In [1]:
import pandas as pd
import numpy as np
import json
from openpyxl import load_workbook, Workbook
import os
import difflib as kwrd
import re
pd.options.display.max_rows = 500

In [2]:
## Assuming you have download Transnational list 2021-04-14.xlsx
## FILE: https://docs.google.com/spreadsheets/d/1zeWnuKsclWILw-LBltuE2q8glsjqg_vE/edit

In [3]:
EXCEL_FILE = 'Transnational list 2021-04-14.xlsx'
EXCEL_TMP_FILE = '{}-tmp.xlsx'.format(EXCEL_FILE)
OUTPUT_FOLDER = "../backend/dev/resources/files/"
country_group_column = {0:'id',1:'label',2:'name'}

In [4]:
def format_sheet(title):
    return 's-{}'.format(title.strip().replace(' ','').lower())

In [5]:
## Create Temporary Clean Version of Excel Sheet
wb = load_workbook(EXCEL_FILE)
for a in wb:
    a.title = format_sheet(a.title)
wb.save(EXCEL_TMP_FILE)

In [6]:
sheet_to_df_map = list(pd.read_excel(EXCEL_TMP_FILE, sheet_name=None))
country_group = pd.read_excel(EXCEL_TMP_FILE, sheet_name=sheet_to_df_map[0])
country_group = country_group[list(country_group)[:3]]
country_group_columns_rename = {}
for i, x in enumerate(list(country_group)[:3]):
    country_group = country_group.rename(columns={x:country_group_column[i]})

In [7]:
country_group['id'] = country_group['id'].fillna(0).astype('int')
country_group['category'] = ''

In [8]:
# indexing category and modify duplicated id
category = ''
current_id = country_group.loc[1,'id']
for i in range(0, len(country_group)):
    if country_group.loc[i, 'name'] != country_group.loc[i, 'name']:
        category = country_group.loc[i, 'label']
    if country_group.loc[i, 'name'] == country_group.loc[i, 'name']:
        country_group.loc[i,'category'] = category
        # remove below code if id is not duplicated anymore
        if current_id < int(country_group.loc[i,'id']):
            country_group.loc[i,'id'] = current_id + 1
        current_id = country_group.loc[i,'id']

In [9]:
# Stip all extra spaces
object_column = list(country_group.select_dtypes(['object']))
country_group[object_column] = country_group[object_column].apply(lambda x: x.str.strip())

In [10]:
country_group = country_group[country_group['name'] == country_group['name']].reset_index()
country_group['sheet_name'] = country_group['name'].apply(lambda x: format_sheet(x))
country_group['countries'] = ''

In [11]:
# creating country_group_countries-new.json
# removing tmp file
country_group_countries = {}
for i in range(0, len(country_group)):
    sheet_name = country_group.loc[i,'sheet_name']
    country_group_name = country_group.loc[i,'name']
    try:
        countries = pd.read_excel(EXCEL_TMP_FILE, sheet_name=sheet_name, names=['name'])
        country_group_countries.update({country_group_name:list(countries['name'])})
    except:
        print('{} doesn\'t exists'.format(sheet_name))
        print('Trying to check matches with removing word "and"')
        try:
            countries = pd.read_excel(EXCEL_TMP_FILE, sheet_name=sheet_name.replace('and',''), names=['name'])
            country_group_countries.update({country_group_name:list(countries['name'])})
            print('{} Found'.format(sheet_name.replace('and','')))
        except:
            print('{} is still Not Found'.format(sheet_name.replace('and','')))

OUTPUT_FILE = "{}country_group_countries-new.json".format(OUTPUT_FOLDER)

with open(OUTPUT_FILE, "w") as f: 
    json.dump(country_group_countries, f,indent=2)
    
os.remove(EXCEL_TMP_FILE)

s-westerncentralandsouthernafrica doesn't exists
Trying to check matches with removing word "and"
s-westerncentralsouthernafrica Found


In [12]:
# We need to confirm to UNEP about type of country group
def get_country_group_type(x):
    return 'region' if 'region' in x.lower() else 'mea'
country_group['type'] = country_group['category'].apply(lambda x: get_country_group_type(x))

In [13]:
## Reshape Country_group
columns = [country_group_column[x] for x in country_group_column]
columns.append('type')
columns.append('category')
country_group = country_group[columns]

In [14]:
## Identification of duplicated with old country_group data

In [15]:
OLD_FILE = "{}country_group.json".format(OUTPUT_FOLDER)
with open(OLD_FILE, "r") as f: 
    old_country_group = json.load(f)

In [16]:
old_country_group = pd.DataFrame(old_country_group)

In [17]:
matched_id = []

def format_match(x):
    convention = re.findall("([a-zA-Z]+\ Convention)", x)
    if len(convention) > 0:
        return convention[0].lower().strip()
    convention = re.findall("[a-zA-Z]+\ Convention", x)
    if len(convention) > 0:
        return convention[0].lower().strip()
    replaces = ['-','(',')','states']
    x = x.lower()
    for r in replaces:
        x = x.replace(r,' ')
    x = x.replace('Asian','Asia')
    return x.strip()

def similar(cg,og):
    cg = cg[~cg.id.isin(matched_id)]
    data = ['id','label','name']
    ##try for exact match
    exact = cg[cg['name'] == og['name']]
    if len(list(exact['name'])) > 0:
        matches = exact[data].to_dict('records')[0]
        matched_id.append(matches['id'])
        return matches
    name = format_match(og['name'].replace('Protocol','Convention'))
    ##try other condition
    if 'convention' in name:
        mg = cg[cg['keywords'] == name]
        if len(list(mg['name'])) > 0:
            matches = mg[data].to_dict('records')[0]
            matched_id.append(matches['id'])
            return matches
        else:
            return None
    keywords = list(cg['keywords'])
    match = kwrd.get_close_matches(name,keywords,n=2, cutoff=0.5) 
    if len(match) > 0:
        matches = cg[cg['keywords'] == match[0]][data].to_dict('records')[0]
        matched_id.append(matches['id'])
        return matches
    return None

country_group['keywords'] = country_group['label'].apply(lambda x: format_match(x))
old_country_group['order'] = old_country_group['name'].apply(lambda x: 1 if "Convention" in x else 2)
old_country_group = old_country_group.sort_values('order')
old_country_group['match'] = old_country_group.apply(lambda x: similar(country_group, x), axis=1)

## Create file to report
old_country_group.to_csv('old-country-group-close-matches-with-new.csv')
country_group[~country_group.id.isin(matched_id)].to_csv('non-duplicated-new-country-group.csv')

In [30]:
print("# Non Duplicated Country Group")
country_group[~country_group.id.isin(matched_id)]

# Non Duplicated Country Group


,id,label,name,type,category,keywords
2,3,Western European and other States,Western European,region,UN Regional Groups of Member States,western european and other
3,4,Eastern European States,Eastern European States,region,UN Regional Groups of Member States,eastern european
13,14,North-East Atlantic (OSPAR) (OSPAR Convention),North-East Atlantic,region,Regional Seas,ospar convention
18,19,ROPME Sea (Kuwait Convention),ROPME Sea,region,Regional Seas,kuwait convention
20,21,South-East Pacific (CPPS) (Lima Convention),South-East Pacific,region,Regional Seas,lima convention
29,30,Convention on Biological Diversity (CBD),Convention on Biological,mea,Global Multi-lateral Environmental Agreements ...,convention on biological diversity cbd
32,33,G7,G7,mea,Transnational Groups of Member States,g7
33,34,G20,G20,mea,Transnational Groups of Member States,g20
34,35,European Union (EU),EU,mea,Transnational Groups of Member States,european union eu
35,36,Commonwealth,Commonwealth,mea,Transnational Groups of Member States,commonwealth


In [33]:
print("# Non Listed Old Country Group")
old_country_group[old_country_group['new_id'] > 1000]

# Non Listed Old Country Group


,id,type,name,order,match,new_id,label
99,100,mea,Water Convention,1,None,1100,Water Convention
26,27,mea,Convention on Biological Diversity,1,None,1027,Convention on Biological Diversity
27,28,mea,Convention on International Trade in Endangere...,1,None,1028,Convention on International Trade in Endangere...
28,29,mea,Convention on Long-range Transboundary Air Pol...,1,None,1029,Convention on Long-range Transboundary Air Pol...
30,31,mea,Convention on the Transboundary Effects of Ind...,1,None,1031,Convention on the Transboundary Effects of Ind...
100,101,mea,World Heritage Convention,1,None,1101,World Heritage Convention
34,35,mea,Espoo Convention,1,None,1035,Espoo Convention
88,89,mea,The London Fisheries Convention,1,None,1089,The London Fisheries Convention
98,99,mea,Waigani Convention,1,None,1099,Waigani Convention
37,38,mea,International Convention for the Prevention of...,1,None,1038,International Convention for the Prevention of...


In [18]:
def json_to_series(text):
    keys, values = zip(*[item for dct in json.loads(text) for item in dct.items()])
    return pd.Series(values, index=keys)

In [19]:
old_country_group['new_id'] = old_country_group.apply(lambda x: x['match']['id'] if x['match'] else x['id'] + 1000,axis=1)

In [20]:
country_group_mapping = {}
for a in old_country_group[['id','new_id']].sort_values('id').to_dict('records'):
    country_group_mapping.update({str(a['id']):str(a['new_id'])})

OUTPUT_FILE = "{}new_country_groups_mapping.json".format(OUTPUT_FOLDER)

with open(OUTPUT_FILE, "w") as f: 
    json.dump(country_group_mapping, f,indent=2)

In [21]:
old_country_group['label'] = old_country_group['name']
old_country_group_to_be_merged = old_country_group[old_country_group['new_id'] > 1000]
old_country_group_to_be_merged = old_country_group_to_be_merged.drop(columns=['id'])
old_country_group_to_be_merged = old_country_group_to_be_merged.rename(columns={'new_id':'id'})
old_country_group_to_be_merged = old_country_group_to_be_merged[['id','label','name','type']]

In [22]:
merged_country_group = pd.concat([country_group,old_country_group_to_be_merged])
merged_country_group = merged_country_group[columns]
merged_country_group['category'] = merged_country_group['category'].apply(lambda x: x if x == x else None)

In [23]:
# creating country_group_countries-new.json
OUTPUT_FILE = "{}country_group-new.json".format(OUTPUT_FOLDER)
merged_country_group.to_json(OUTPUT_FILE,orient='records',indent=2)

In [24]:
# Merging old country group countries with new country group countries

OLD_FILE = "{}country_group_countries.json".format(OUTPUT_FOLDER)
with open(OLD_FILE, "r") as f: 
    old_country_group_countries = json.load(f)
    
NEW_FILE = "{}country_group_countries-new.json".format(OUTPUT_FOLDER)
with open(NEW_FILE, "r") as f: 
    new_country_group_countries = json.load(f)
    
for ocg in old_country_group_to_be_merged['label'].to_list():
    if ocg not in list(old_country_group_countries):
        print("{} doesn't have country group list".format(ocg))
    else:
        new_country_group_countries.update({ocg:old_country_group_countries[ocg]})

OUTPUT_FILE = "{}country_group_countries-new.json".format(OUTPUT_FOLDER)

with open(OUTPUT_FILE, "w") as f: 
    json.dump(new_country_group_countries, f,indent=2)